In [2]:
%reload_ext autoreload
%autoreload 2

import os
import sys
from pathlib import Path

dir_path = Path(os.getcwd()).absolute()
module_path = str(dir_path.parent.parent.parent)  # repo root
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
import pandas as pd
import torch, lifelines

import pytorch_lightning as pl

pl.utilities.seed.seed_everything(seed=42)

import matplotlib.pyplot as plt
from pytorch_lightning import Trainer

# OWN MODULES
from organsync.experiments.data.data_module import UKRegDataModule
from organsync.models.organsync_network import OrganSync_Network
from organsync.models.organite_network import OrganITE_Network
from organsync.models.inference import Inference_OrganITE, Inference_OrganSync

import warnings

warnings.filterwarnings("ignore")

Global seed set to 42


In [3]:
# SETUP DATA

batch_size = 256

root_data_dir = Path("../datasets").absolute()

data_dir = (
    root_data_dir / "processed_UKReg"
)  # replace with your path to the processed UKReg dataset

dm = UKRegDataModule(data_dir, batch_size=batch_size, control=False)
dm.prepare_data()
dm.setup(stage="fit")

dm._train_processed

,PRIMARY_LIVER_DISEASE,reg_age,SERUM_CREATININE,SERUM_BILIRUBIN,INR,SERUM_SODIUM,DIABETIC,regyr,rwtime,CENS,...,DCOD_3,DGRP_1,DGRP_2,DGRP_3,DGRP_4,DBG_1.0,DBG_2.0,DBG_3.0,DBG_4.0,Y
4672,-1.143617,-0.124023,-0.101377,0.248751,0.447170,-0.423979,-0.408407,-0.888355,-0.419680,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.603575
10285,0.000063,1.064747,-0.575057,0.471697,-0.165238,0.219349,-0.408407,0.353916,0.274812,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.337767
16627,1.524971,1.234571,-0.490643,-0.415297,-0.845485,0.835131,-0.408407,1.320128,-0.695434,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.907274
10579,0.381290,1.489308,1.155433,1.966445,-0.165238,-0.806955,-0.408407,0.353916,-0.639262,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.901996
13750,0.381290,0.894923,-0.490643,-0.136058,-1.072233,0.219349,-0.408407,0.906037,-0.475852,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.274915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11284,-0.762390,0.215626,0.627845,-0.579555,-0.391987,0.424609,2.448540,0.491947,0.039910,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.838185
11964,1.143744,1.234571,-1.060439,-0.587768,-0.165238,0.424609,-0.408407,0.629977,0.233959,1,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-0.819953
5390,0.381290,1.489308,-0.385125,0.471697,-0.618736,-0.806955,-0.408407,-0.750325,1.561664,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.391989
860,1.524971,0.385450,1.106616,0.501615,-0.426940,0.023156,-0.408407,-1.716536,-0.634156,0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.714879


# OrganITE

In [6]:
# OrganITE

input_dim = dm.size(1)
hidden_dim = 16
num_hidden_layers = 4
output_dim = 6
lr = 0.007
gamma = 0.79
lambd = 0.15
kappa = 0.15
weight_decay = 0.0006
n_clusters = 15
activation_type = "leaky_relu"
dropout_prob = 0.11
epochs = 10

organite_model = OrganITE_Network(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_hidden_layers=num_hidden_layers,
    output_dim=output_dim,
    lr=lr,
    gamma=gamma,
    lambd=lambd,
    kappa=kappa,
    weight_decay=weight_decay,
    n_clusters=n_clusters,
    activation_type=activation_type,
    dropout_prob=dropout_prob,
).double()

dm.setup(stage="fit")
organite_trainer = Trainer(callbacks=[], max_epochs=epochs)
organite_trainer.fit(organite_model, datamodule=dm)

dm.setup(stage="test")
organite_trainer.test(datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type             | Params
----------------------------------------------------
0 | representation | Sequential       | 2.0 K 
1 | propensity     | Sequential       | 147   
2 | output         | Sequential       | 49    
3 | loss_mse       | MeanSquaredError | 0     
4 | loss_cel       | CrossEntropyLoss | 0     
----------------------------------------------------
2.2 K     Trainable params
0         Non-trainable params
2.2 K     Total params
0.009     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_loss - RMSE': 0.49413105845451355,
 'test_loss - mean difference in days': 645.9779052734375}
--------------------------------------------------------------------------------


[{'test_loss - mean difference in days': 645.9779052734375,
  'test_loss - RMSE': 0.49413105845451355}]

In [7]:
import cloudpickle

organite_model.eval()
inference_oite = Inference_OrganITE(model=organite_model, mean=dm.mean, std=dm.std)

del organite_model.trainer
organite_model.trainer = None

with open("models/organite_inference.p", "wb") as f:
    cloudpickle.dump(inference_oite, f)

# OrganSync

In [ ]:
# OrganSync


lr = 0.0095
weight_decay = 0.00001
num_hidden_layers = 3
hidden_dim = 50
output_dim = 19
dropout_prob = 0.006
epochs = 100
batch_size = 128
activation_type = "leaky_relu"
gamma = 0.87  # lr decay
lambd = 0.1

# CONSTRUCT MODEL(W)
input_dim = dm.size(1)
organsync_model_with_organ = OrganSync_Network(
    input_dim=input_dim,
    hidden_dim=hidden_dim,
    num_hidden_layers=num_hidden_layers,
    output_dim=output_dim,
    lr=lr,
    gamma=gamma,
    lambd=lambd,
    weight_decay=weight_decay,
    activation_type=activation_type,
    dropout_prob=dropout_prob,
).double()

trainer_with_organ = Trainer(callbacks=[], max_epochs=epochs)
trainer_with_organ.fit(organsync_model_with_organ, datamodule=dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type             | Params
----------------------------------------------------
0 | representation | Sequential       | 16.4 K
1 | beta           | Linear           | 20    
2 | loss           | MeanSquaredError | 0     
----------------------------------------------------
16.4 K    Trainable params
0         Non-trainable params
16.4 K    Total params
0.066     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Global seed set to 42


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
import cloudpickle

organsync_model_with_organ.eval()
del organsync_model_with_organ.trainer
organsync_model_with_organ.trainer = None

inference_os = Inference_OrganSync(
    model=organsync_model_with_organ, mean=dm.mean, std=dm.std
)


with open("models/organsync_inference.p", "wb") as f:
    cloudpickle.dump(inference_os, f)

inference_os

# OrganITE VAE

In [ ]:
# Organ clustering
test_df = dm._train_processed

organs = test_df[list(dm.o_cols)]

In [ ]:
probs = np.bincount(organite_model.cluster.predict(organs)) / len(organs) * 100

probs

In [ ]:
import cloudpickle


with open("models/organite_clustering_probs.p", "wb") as f:
    cloudpickle.dump(probs, f)

organs.to_csv("models/organs.csv", compression="gzip", index=None)